# Interactive Normalization Methods Demonstration

This interactive tool demonstrates how different normalization methods transform student scores.

**Instructions:**
1. Use the **preset buttons** to load example score distributions
2. Adjust individual student scores with the **sliders**
3. Watch how the **six normalization panels** respond in real-time

**Key observations to explore:**
- How do tied scores behave differently across methods?
- What happens when you move a student across a grade boundary (e.g., 90 → 91)?
- How does changing one extreme score affect the others in z-score vs. other methods?
- Which methods distinguish between students with close scores in the A range?
- **Toggle the checkbox** to see what happens when normalization methods (panels 3-6) are applied to already-bucketed GPA instead of raw scores
- **Switch between Mean/Median** to see how the choice of center measure affects Z-score and Delta Norm (panels 5-6)

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Diagnostic cell - skip in Voila (uncomment to debug widget issues in JupyterLab)
# import sys, importlib
# import ipywidgets as widgets
# print("Python:", sys.version.split()[0])
# print("ipywidgets:", widgets.__version__)
# import jupyterlab
# print("jupyterlab:", jupyterlab.__version__)
# print("jupyterlab_widgets installed:", importlib.util.find_spec("jupyterlab_widgets") is not None)
# print("widgetsnbextension installed:", importlib.util.find_spec("widgetsnbextension") is not None)
# widgets.IntSlider()  # Render test
# print("voila installed:", importlib.util.find_spec("voila") is not None)

Python: 3.11.14
ipywidgets: 8.1.8
jupyterlab: 4.5.0
jupyterlab_widgets installed: True
widgetsnbextension installed: True
voila installed: True


In [3]:
# Core computation functions

def compute_normalized_rank(scores):
    """Compute normalized rank: r = (rank - 1) / (N - 1)"""
    N = len(scores)
    ranks = stats.rankdata(scores, method='average')
    normalized_ranks = (ranks - 1) / (N - 1)
    return normalized_ranks

def compute_zscore(scores, use_median=False):
    """Compute z-scores: z = (x - center) / σ
    
    Args:
        scores: Array of scores
        use_median: If True, use median as center; if False, use mean
    """
    scores_array = np.array(scores)
    
    # Check if all scores are identical (std deviation = 0 or very close)
    if np.std(scores_array) < 1e-10:
        # Return zeros for all identical values
        return np.zeros(len(scores_array))
    
    if use_median:
        # Use median as center instead of mean
        center = np.median(scores_array)
        std = np.std(scores_array)
        return (scores_array - center) / std
    else:
        # Standard z-score with mean
        return stats.zscore(scores_array)

def compute_delta_norm(scores, use_median=True):
    """Compute delta from center: δ = (x - center)
    
    Args:
        scores: Array of scores
        use_median: If True, use median as center; if False, use mean
    """
    scores_array = np.array(scores)
    if use_median:
        return scores_array - np.median(scores_array)
    else:
        return scores_array - np.mean(scores_array)

def bucket_to_gpa(raw_score):
    """Convert raw score to bucketed GPA using traditional letter grade cutoffs."""
    if raw_score >= 91:
        return 4.0
    elif raw_score >= 81:
        return 3.67
    elif raw_score >= 71:
        return 3.33
    elif raw_score >= 61:
        return 3.0
    elif raw_score >= 51:
        return 2.67
    elif raw_score >= 41:
        return 2.33
    elif raw_score >= 31:
        return 2.0
    elif raw_score >= 21:
        return 1.67
    elif raw_score >= 11:
        return 1.33
    else:
        return 1.0

def compute_precision_gpa(raw_score):
    """Compute Precision GPA using within-bin linear interpolation."""
    bins = [
        (90, 100, 3.67, 4.0),   # A
        (80, 90,  3.33, 3.67),  # A-
        (70, 80,  3.0,  3.33),  # B+
        (60, 70,  2.67, 3.0),   # B
        (50, 60,  2.33, 2.67),  # B-
        (40, 50,  2.0,  2.33),  # C+
        (30, 40,  1.67, 2.0),   # C
        (20, 30,  1.33, 1.67),  # C-
        (10, 20,  1.0,  1.33),  # D+
        (0,  10,  0.0,  1.0),   # D
    ]
    
    for low, high, gpa_low, gpa_high in bins:
        if low < raw_score <= high:
            offset = (raw_score - low) / (high - low)
            pgpa = offset * (gpa_high - gpa_low) + gpa_low
            return pgpa
    
    return 0.0

def prepare_plot_data(raw_scores, use_bucketed=False, use_median=False):
    """Compute all normalization methods.
    
    Args:
        raw_scores: List of raw scores
        use_bucketed: If True, compute normalization methods on bucketed GPA instead of raw scores
        use_median: If True, use median for centering; if False, use mean
    """
    student_ids = [f"S{i+1}" for i in range(len(raw_scores))]
    
    df = pd.DataFrame({
        'student_id': student_ids,
        'raw_score': raw_scores,
        'raw_normalized': [score / 100.0 for score in raw_scores]
    })
    
    # Bucketed GPA (always computed from raw scores)
    df['bucketed_gpa'] = df['raw_score'].apply(bucket_to_gpa)
    
    # Choose base for normalization methods
    base_scores = df['bucketed_gpa'] if use_bucketed else df['raw_score']
    
    # Normalized rank
    df['norm_rank'] = compute_normalized_rank(base_scores)
    
    # Precision GPA (only meaningful from raw scores)
    if use_bucketed:
        # When using bucketed, precision GPA is same as bucketed
        df['precision_gpa'] = df['bucketed_gpa']
    else:
        df['precision_gpa'] = df['raw_score'].apply(compute_precision_gpa)
    
    # Z-score
    df['zscore'] = compute_zscore(base_scores, use_median=use_median)
    
    # Scale z-scores to [0, 1] for plotting using fixed theoretical range
    # Use -3 to +3 as standard z-score range (covers ~99.7% of normal distribution)
    z_min_theoretical = -3
    z_max_theoretical = 3
    z_range_theoretical = z_max_theoretical - z_min_theoretical
    df['zscore_scaled'] = (df['zscore'] - z_min_theoretical) / z_range_theoretical
    
    # Clip to [0, 1] in case actual values exceed -3 to +3 range
    df['zscore_scaled'] = df['zscore_scaled'].clip(0, 1)
    
    # Store both theoretical range and actual range for display
    df.attrs['z_min'] = z_min_theoretical
    df.attrs['z_max'] = z_max_theoretical
    df.attrs['z_actual_min'] = df['zscore'].min()
    df.attrs['z_actual_max'] = df['zscore'].max()
    
    # Delta Norm
    df['delta_norm'] = compute_delta_norm(base_scores, use_median=use_median)
    
    # Scale delta norms to [0, 1] for plotting using fixed theoretical range
    if use_bucketed:
        # Bucketed GPA range: 0-4, so delta can be -4 to +4
        d_min_theoretical = -4
        d_max_theoretical = 4
    else:
        # Raw scores are 0-100, so delta from median can range from -100 to +100
        d_min_theoretical = -100
        d_max_theoretical = 100
    
    d_range_theoretical = d_max_theoretical - d_min_theoretical
    df['delta_norm_scaled'] = (df['delta_norm'] - d_min_theoretical) / d_range_theoretical
    
    df.attrs['d_min'] = d_min_theoretical
    df.attrs['d_max'] = d_max_theoretical
    df.attrs['use_bucketed'] = use_bucketed
    df.attrs['use_median'] = use_median
    
    return df

In [ ]:
# Plotting function

def plot_normalization_methods(raw_scores, use_bucketed=False, use_median=False):
    """Create the 6-panel visualization."""
    df = prepare_plot_data(raw_scores, use_bucketed, use_median)
    
    # Create figure
    fig, axes = plt.subplots(1, 6, figsize=(21, 4))
    
    # Color mapping based on raw scores
    norm = Normalize(vmin=30, vmax=100)
    cmap = plt.cm.viridis
    colors = [cmap(norm(score)) for score in df['raw_score']]
    
    # Common parameters
    point_size = 120
    point_alpha = 0.85
    x_spacing = 0.5
    x_positions = np.arange(len(df)) * x_spacing + 1
    
    # Suffix for title indicating what the methods are based on
    # Keep it short to avoid layout shifts
    base_suffix = " [bucketed]" if use_bucketed else " [raw]"
    center_label = "median" if use_median else "mean"
    
    # Panel 1: Raw Score / 100
    ax = axes[0]
    ax.scatter(x_positions, df['raw_normalized'], 
               c=colors, s=point_size, alpha=point_alpha, 
               edgecolors='black', linewidths=1.5, zorder=3)
    
    for i, row in df.iterrows():
        ax.text(x_positions[i], row['raw_normalized'] + 0.03, 
                f"{row['student_id']}\n{row['raw_score']}", 
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylim(-0.05, 1.15)
    ax.set_xlim(0.4, x_positions[-1] + 0.6)
    ax.set_ylabel('Raw Score', fontsize=11)
    ax.set_xlabel('Students', fontsize=11)
    ax.set_title('Raw Score / 100', fontsize=12, fontweight='bold')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['0', '25', '50', '75', '100'])
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks([])
    
    # Panel 2: Bucketed GPA
    ax = axes[1]
    bucketed_scaled = df['bucketed_gpa'] / 4.0
    
    ax.scatter(x_positions, bucketed_scaled, 
               c=colors, s=point_size, alpha=point_alpha, 
               edgecolors='black', linewidths=1.5, zorder=3)
    
    for i, row in df.iterrows():
        ax.text(x_positions[i], bucketed_scaled.iloc[i] + 0.03, 
                f"{row['student_id']}\n{row['bucketed_gpa']:.2f}", 
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylim(-0.05, 1.15)
    ax.set_xlim(0.4, x_positions[-1] + 0.6)
    ax.set_ylabel('GPA', fontsize=11)
    ax.set_xlabel('Students', fontsize=11)
    ax.set_title('Bucketed GPA\n(letter grade bins)', fontsize=12, fontweight='bold')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['0.0', '1.0', '2.0', '3.0', '4.0'])
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks([])
    
    # Panel 3: Precision GPA
    ax = axes[2]
    pgpa_scaled = df['precision_gpa'] / 4.0
    
    ax.scatter(x_positions, pgpa_scaled, 
               c=colors, s=point_size, alpha=point_alpha, 
               edgecolors='black', linewidths=1.5, zorder=3)
    
    for i, row in df.iterrows():
        ax.text(x_positions[i], pgpa_scaled.iloc[i] + 0.03, 
                f"{row['student_id']}\n{row['precision_gpa']:.2f}", 
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylim(-0.05, 1.15)
    ax.set_xlim(0.4, x_positions[-1] + 0.6)
    ax.set_ylabel('GPA', fontsize=11)
    ax.set_xlabel('Students', fontsize=11)
    ax.set_title(f'Precision GPA\n(within-bin interpolation){base_suffix}', fontsize=12, fontweight='bold')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['0.0', '1.0', '2.0', '3.0', '4.0'])
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks([])
    
    # Panel 4: Normalized Rank
    ax = axes[3]
    
    ax.scatter(x_positions, df['norm_rank'], 
               c=colors, s=point_size, alpha=point_alpha, 
               edgecolors='black', linewidths=1.5, zorder=3)
    
    for i, row in df.iterrows():
        ax.text(x_positions[i], row['norm_rank'] + 0.03, 
                f"{row['student_id']}\n{row['norm_rank']:.2f}", 
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylim(-0.05, 1.15)
    ax.set_xlim(0.4, x_positions[-1] + 0.6)
    ax.set_ylabel('Normalized Rank', fontsize=11)
    ax.set_xlabel('Students', fontsize=11)
    ax.set_title(f'Normalized Rank\n(rank−1)/(N−1){base_suffix}', fontsize=12, fontweight='bold')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['0.0', '0.25', '0.5', '0.75', '1.0'])
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks([])
    
    # Panel 5: Z-score
    ax = axes[4]
    z_min = df.attrs['z_min']
    z_max = df.attrs['z_max']
    
    # Plot each point with appropriate marker based on whether it's clipped
    for i, row in df.iterrows():
        z_value = row['zscore']
        
        # Choose marker based on whether value is outside display range
        if z_value < z_min:
            marker = 'v'  # Triangle down for values below range
        elif z_value > z_max:
            marker = '^'  # Triangle up for values above range
        else:
            marker = 'o'  # Circle for values in range
        
        ax.scatter(x_positions[i], row['zscore_scaled'], 
                   c=[colors[i]], s=point_size, alpha=point_alpha, 
                   marker=marker, edgecolors='black', linewidths=1.5, zorder=3)
    
    # Add text labels
    for i, row in df.iterrows():
        ax.text(x_positions[i], row['zscore_scaled'] + 0.03, 
                f"{row['student_id']}\n{row['zscore']:.2f}", 
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylim(-0.05, 1.15)
    ax.set_xlim(0.4, x_positions[-1] + 0.6)
    ax.set_ylabel('Z-score', fontsize=11)
    ax.set_xlabel('Students', fontsize=11)
    ax.set_title(f'Z-score\n(x−{center_label})/σ{base_suffix}', fontsize=12, fontweight='bold')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['-3', '-1.5', '0', '1.5', '3'])
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks([])
    
    # Add text annotations showing theoretical and actual z-score ranges
    z_actual_min = df.attrs['z_actual_min']
    z_actual_max = df.attrs['z_actual_max']
    ax.text(0.98, 0.08, f'Display range: [{z_min:.0f}, {z_max:.0f}]',
            transform=ax.transAxes, ha='right', va='bottom',
            fontsize=8, style='italic', color='gray')
    ax.text(0.98, 0.02, f'Actual range: [{z_actual_min:.2f}, {z_actual_max:.2f}]',
            transform=ax.transAxes, ha='right', va='bottom',
            fontsize=8, style='italic', color='gray')
    
    # Panel 6: Delta Norm
    ax = axes[5]
    d_min = df.attrs['d_min']
    d_max = df.attrs['d_max']
    
    ax.scatter(x_positions, df['delta_norm_scaled'], 
               c=colors, s=point_size, alpha=point_alpha, 
               edgecolors='black', linewidths=1.5, zorder=3)
    
    for i, row in df.iterrows():
        ax.text(x_positions[i], row['delta_norm_scaled'] + 0.03, 
                f"{row['student_id']}\n{row['delta_norm']:.1f}", 
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylim(-0.05, 1.15)
    ax.set_xlim(0.4, x_positions[-1] + 0.6)
    ax.set_ylabel('Delta Norm', fontsize=11)
    ax.set_xlabel('Students', fontsize=11)
    ax.set_title(f'Delta Norm\n(x−{center_label}){base_suffix}', fontsize=12, fontweight='bold')
    
    # Set y-ticks based on the theoretical range (changes with bucketed toggle)
    d_range = d_max - d_min
    tick_values = [d_min, d_min + 0.25*d_range, d_min + 0.5*d_range, 
                   d_min + 0.75*d_range, d_max]
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels([f'{v:.0f}' if abs(v) >= 10 else f'{v:.1f}' for v in tick_values])
    
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks([])
    
    # Use fixed spacing to prevent layout shifts when toggling
    plt.subplots_adjust(left=0.05, right=0.98, top=0.92, bottom=0.15, wspace=0.3)
    
    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar_ax = fig.add_axes([0.2, 0.02, 0.6, 0.02])
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Raw Score (colors track students across panels)', fontsize=10)
    
    # Display figure and close (avoid plt.show() which can block)
    display(fig)
    plt.close(fig)


In [5]:
# Preset score distributions

PRESETS = {
    'Wide Range': [32, 45, 67, 67, 81, 81, 81, 88, 91, 92, 97, 98],
    'High Scores': [85, 87, 88, 88, 89, 91, 93, 94, 96, 97, 99, 99],
    'Low Scores': [35, 37, 38, 38, 39, 41, 43, 44, 46, 47, 49, 49],
    'Letter grades only': [40, 50, 60, 70, 80, 80, 90, 90, 90, 100, 100, 100],
    'All Tied': [85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85],
    'Bimodal': [65, 66, 67, 68, 69, 92, 93, 94, 95, 96, 97, 98],
    'Evenly Spaced': [50, 54, 58, 62, 66, 70, 74, 78, 82, 86, 90, 94],
    'One Outlier': [88, 89, 89, 90, 90, 91, 91, 92, 92, 93, 93, 32],
}

NUM_STUDENTS = 12

## Interactive Controls

Click a preset button below, then adjust individual student scores using the sliders. The visualization updates automatically.


In [ ]:
# Create interactive interface

# Create sliders for each student
sliders = []
for i in range(NUM_STUDENTS):
    slider = widgets.IntSlider(
        value=85,
        min=30,
        max=100,
        step=1,
        description=f'S{i+1}:',
        continuous_update=False,  # Only update when released
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style={'description_width': '35px'},
        layout=widgets.Layout(width='280px')
    )
    sliders.append(slider)

# Create output widget for the plot
output = widgets.Output()

# Create toggle for bucketed mode
bucketed_toggle = widgets.Checkbox(
    value=False,
    description='Use Bucketed GPA as base for normalization methods',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Radio buttons for center measure (mean vs median)
center_measure_radio = widgets.RadioButtons(
    options=['Mean', 'Median'],
    value='Mean',
    description='Center measure for Z-score & Delta Norm:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Update function
def update_plot(*args):
    """Update the plot when sliders, toggle, or radio buttons change."""
    raw_scores = [s.value for s in sliders]
    use_bucketed = bucketed_toggle.value
    use_median = (center_measure_radio.value == 'Median')
    with output:
        clear_output(wait=True)
        plot_normalization_methods(raw_scores, use_bucketed, use_median)

# Attach update function to all sliders, toggle, and radio buttons
for slider in sliders:
    slider.observe(update_plot, names='value')
bucketed_toggle.observe(update_plot, names='value')
center_measure_radio.observe(update_plot, names='value')

# Preset buttons
preset_buttons = {}  # Change to dict to track buttons by name
active_preset = None

def load_preset(preset_name):
    """Load a preset score distribution."""
    global active_preset
    scores = PRESETS[preset_name]
    
    # Update button styles to highlight active preset
    for name, button in preset_buttons.items():
        if name == preset_name:
            button.button_style = 'success'  # Green for active
        else:
            button.button_style = ''  # Default grey for inactive
    
    active_preset = preset_name
    
    # Temporarily disconnect observers to avoid multiple redraws
    for slider in sliders:
        slider.unobserve(update_plot, names='value')
    
    # Set all slider values
    for i, score in enumerate(scores):
        sliders[i].value = score
    
    # Reconnect observers
    for slider in sliders:
        slider.observe(update_plot, names='value')
    
    # Update plot once
    update_plot()

for name in PRESETS.keys():
    button = widgets.Button(
        description=name,
        button_style='',  # Start with default grey
        layout=widgets.Layout(width='150px', margin='2px')
    )
    button.on_click(lambda b, n=name: load_preset(n))
    preset_buttons[name] = button

# Layout
preset_box = widgets.HBox(
    list(preset_buttons.values()),
    layout=widgets.Layout(justify_content='center', margin='10px 0px')
)

# Arrange sliders in 3 columns
col1 = widgets.VBox(sliders[0:4])
col2 = widgets.VBox(sliders[4:8])
col3 = widgets.VBox(sliders[8:12])
slider_box = widgets.HBox(
    [col1, col2, col3],
    layout=widgets.Layout(justify_content='center', margin='10px 0px')
)

# Display everything
display(preset_box)
display(slider_box)
display(bucketed_toggle)
display(center_measure_radio)
display(output)

# Trigger initial plot after page loads using JavaScript
# This avoids blocking during Voila's pre-render phase
from IPython.display import Javascript
display(Javascript('''
    // Wait for widgets to be ready, then click the first preset button
    setTimeout(function() {
        var buttons = document.querySelectorAll('button.jupyter-button');
        for (var btn of buttons) {
            if (btn.textContent.includes('Wide Range')) {
                btn.click();
                break;
            }

        }
    }, 500);
'''))



Checkbox(value=False, description='Use Bucketed GPA as base for normalization methods', layout=Layout(width='4…

RadioButtons(description='Center measure for Z-score & Delta Norm:', layout=Layout(width='400px'), options=('M…

Output()

## Notes

### Key Observations

**Bucketed GPA (Panel 2):**
- Creates "plateaus" where students with different raw scores get identical GPAs
- Sharp jumps at grade boundaries (e.g., 90 → 91 jumps from 3.67 to 4.0)
- Try the "High Scores" preset to see many students bunched at 4.0

**Precision GPA (Panel 3):**
- Maintains grade bins but distinguishes students *within* each bin
- Still has jumps between bins, but smoother within bins
- Compare with Bucketed GPA in the "High Scores" preset

**Normalized Rank (Panel 4):**
- Perfectly spreads students from 0 to 1
- Tied scores get the same rank (average of their positions)
- Try "All Tied" to see everyone collapse to 0.5

**Z-score (Panel 5):**
- Depends on the mean and standard deviation
- Moving *any* student affects *everyone's* z-score
- Try "One Outlier" and move the outlier (S12) around
- "All Tied" gives undefined z-scores (division by zero)
- **Triangle markers** indicate values outside the [-3, +3] display range: ▼ for below, ▲ for above

### Experimental Scenarios

1. **Grade Inflation**: Use "High Scores" and observe how different methods handle top-heavy distributions

2. **Grade Deflation**: Use "Low Scores" to see how methods behave with bottom-heavy distributions where most students fail

3. **Boundary Effects**: Start with "Wide Range" and move a student from 90 to 91. Watch the jump in Bucketed GPA.

4. **Tied Scores**: Use "All Tied" to see which methods can handle ties gracefully

5. **Outlier Sensitivity**: Use "One Outlier" and move S12. Notice how z-scores change for *everyone*

6. **Within-bin Discrimination**: Use "High Scores" and compare panels 2 and 3 to see how Precision GPA distinguishes students in the A range

7. **No Precision Gain**: Use "Letter grades only" where all scores fall on bin boundaries. Panels 2 and 3 (Bucketed GPA and Precision GPA) become identical since there's no within-bin variation to capture

8. **Normalizing Already-Bucketed Data**: Check the toggle box to compute normalization methods from Bucketed GPA instead of raw scores. Notice how:
   - Panel 3 (Precision GPA) becomes identical to Panel 2 (can't add precision to already-bucketed data)
   - Panel 4 (Normalized Rank) may show more ties since multiple raw scores map to same GPA
   - Panels 5 & 6 (Z-score & Delta Norm) operate on the coarser 0-4 scale instead of 0-100

9. **Mean vs Median Center**: Use "One Outlier" and toggle between Mean and Median. Compare panels 5 (Z-score) and 6 (Delta Norm):
   - With **Mean**: The outlier (S12) pulls the center, affecting all scores
   - With **Median**: The center is more robust to the outlier, so other scores are less affected
